In [19]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json
import jsonlines
import itertools


In [2]:
NER = ["EFFECT"]

In [3]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [4]:
df = df.reset_index().drop(columns=["index"])

In [5]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Some, isolates, such, as, CD196, have, high, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-1.972253084182739, -2.1824533939361572, 3.4...","[-100, 2, 2, 2, 2, 2, -100, 2, 2, 2, -100, 2, ...","[None, 0, 1, 2, 3, 4, 4, 5, 6, 7, 7, 8, 9, 10,...",EFFECT
1,"[The, type, strain, ,, Sanger_04^T, (, =DSM, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,"[[-1.9760128259658811, -2.7345712184906, 4.394...","[-100, 2, 2, 2, 2, 2, -100, -100, -100, -100, ...","[None, 0, 1, 2, 3, 4, 4, 4, 4, 4, 4, 5, 6, 6, ...",EFFECT
2,"[IMP, was, almost, exclusively, found, in, P.,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-2.040075540542602, -2.656398296356201, 4.31...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 11...",EFFECT
3,"[Klebsiella, oxytoca, ferments, ferric, citrat...","[O, O, B, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-1.697213888168335, -2.056787967681884, 3.15...","[-100, 2, 2, -100, 0, -100, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10,...",EFFECT
4,"[Typically, ,, each, Hymenoscyphus, species, i...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-1.917844057083129, -2.2763607501983643, 3.9...","[-100, 2, 2, 2, 2, -100, -100, -100, -100, -10...","[None, 0, 1, 2, 3, 3, 3, 3, 3, 3, 4, 5, 6, 7, ...",EFFECT
...,...,...,...,...,...,...,...
618,"[The, J_IIIs, were, cultured, with, the, fungu...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",618,"[[-2.135071039199829, -2.359669685363769, 3.66...","[-100, 2, 2, -100, -100, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, ...",EFFECT
619,"[From, this, ,, we, obtained, a, unialgal, iso...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",619,"[[-2.089053869247436, -2.699814319610595, 4.29...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, 9, ...",EFFECT
620,"[Genomic, DNA, was, extracted, from, N., varia...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",620,"[[-2.488898038864135, -2.677292585372925, 4.68...","[-100, 2, 2, 2, 2, 2, 2, -100, 2, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 5, 6, 6, 7, 8, 8, 9, ...",EFFECT
621,"[The, 12.2, kb, LPS, gene, cluster, of, BXO1, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",621,"[[-2.260250568389892, -2.606179475784301, 4.59...","[-100, 2, 2, -100, -100, 2, 2, 2, 2, 2, 2, -10...","[None, 0, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, ...",EFFECT


In [6]:
label_ids = df[["label_ids"]]

In [7]:
annotations = []
current_annotation = None

for j in range(len(df)):
	for i in zip(df.loc[j, "tokens"], df.loc[j, "ner_tags"]):
		if i[1] != "O":
			if current_annotation is None:
				current_annotation = { "text": i[0]}
			else:
				current_annotation["text"] += " " + i[0]
		else:
			if current_annotation is not None:
				annotations.append(current_annotation)
				current_annotation = None

if current_annotation is not None:
	annotations.append(current_annotation)


### Predict sentences again

In [8]:
ner = "EFFECT"

In [9]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=0)


In [10]:
# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:
            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

Some isolates such as CD196 have high BSR values for the CdeA protein but are susceptible to some fluoroquinolones (gatifloxacin, moxifloxacin, levofloxacin [ 2 ]), indicating the presence of this gene alone does not infer resistance to newer fluoroquinolones in ST1 genomes.
[]
The type strain, Sanger_04^T (=DSM 102317^T), was isolated from human faeces.
[]
IMP was almost exclusively found in P. aeruginosa isolates, showing the importance of this species in the dissemination of this carbapenemase; it was associated with different clusters in a single hospital and each variant was associated with a specific clone: IMP-8/ST348, IMP-23/ST175 and IMP-16/ST253.
[]
Klebsiella oxytoca ferments ferric citrate under anaerobic conditions and produces a ferric hydrogel, whereas ferrous ions were formed in solution.
[]
Typically, each Hymenoscyphus species is predicted to secrete three to four S1/P1 nucleases, one Ribonuclease 2-5A and three close homologues of T2 RNAse family protein.
[]
The stra

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[]
The strains “Methylacidiphilum fumariolicum” SolV and “Methylacidiphilum infernorum” V4 possess three complete pmoCAB operons, while “Methylacidiphilum kamchatkense” Kam1 has a fourth distinct copy of pmoA, localized in a truncated pmoCA operon.
[]
Easson et al. described spontaneous mutations and genetic instability of the strain Shinella in the region encoding the cellulose biosynthetic cluster [16].
[]
In a recent study, an excellent plant inter-rhizosphere strain, B. aryabhattai LAD, was isolated from maize rhizosphere, and the impact of LAD on rhizosphere microbial structure was also explored in the previous study ( Deng et al., 2022 ).
[{'entity_group': 'B', 'score': 0.64810354, 'word': 'plant', 'start': 32, 'end': 37}, {'entity_group': 'I', 'score': 0.54434437, 'word': 'inter - rhizosphere', 'start': 38, 'end': 55}]
Strain 1980 or Ss-1 was originally isolated from bean culls in western Nebraska by Dr.
[]
Further, the presence of xopAQ was predicted in the genomes of X. hortor

In [11]:
predictions = [p for p in predictions if p]

In [12]:
predictions

[[{'entity_group': 'B',
   'score': 0.8598016,
   'word': 'antimicrobial',
   'start': 26,
   'end': 39}],
 [{'entity_group': 'B',
   'score': 0.64810354,
   'word': 'plant',
   'start': 32,
   'end': 37},
  {'entity_group': 'I',
   'score': 0.54434437,
   'word': 'inter - rhizosphere',
   'start': 38,
   'end': 55}],
 [{'entity_group': 'B',
   'score': 0.7079398,
   'word': 'dysregulation',
   'start': 56,
   'end': 69},
  {'entity_group': 'I',
   'score': 0.6500979,
   'word': 'of t1ifn expression',
   'start': 70,
   'end': 89},
  {'entity_group': 'B',
   'score': 0.40742454,
   'word': 'autocrine',
   'start': 94,
   'end': 103},
  {'entity_group': 'I',
   'score': 0.8620253,
   'word': 'signaling',
   'start': 104,
   'end': 113}],
 [{'entity_group': 'B',
   'score': 0.46547407,
   'word': 'phagocytosis',
   'start': 115,
   'end': 127},
  {'entity_group': 'B',
   'score': 0.44186425,
   'word': 'phagocytosis',
   'start': 229,
   'end': 241}],
 [{'entity_group': 'I',
   'score': 

In [13]:
annotations

[{'text': 'ferments'},
 {'text': 'antimicrobial'},
 {'text': 'spontaneous mutations'},
 {'text': 'genetic instability'},
 {'text': 'blue discoloration of soy foods'},
 {'text': 'blue discoloration'},
 {'text': 'ruminal fermentation'},
 {'text': 'digestibility of nutrients'},
 {'text': 'N flows in a dual-flow continuous culture system fed diets formulated'},
 {'text': 'photoinactivation'},
 {'text': 'hampering lipolysis and mitochondrial β-oxidation'},
 {'text': 'remove 99.0 % of the pollutant'},
 {'text': 'Al tolerance'},
 {'text': 'negative phototaxis to UV light'},
 {'text': 'reduction in the number of infected cases'},
 {'text': 'appearance of new variants with different antigenicities'},
 {'text': 'increased infectivity'},
 {'text': 'higher pathogenicity'},
 {'text': 'apoptosis'},
 {'text': 'amino acid uptake deficiencies'},
 {'text': 'gene expansion'},
 {'text': 'synthesis and degradation'},
 {'text': 'retrieve a carbon source from glycogen storage'},
 {'text': 'normal growth'},
 

In [14]:
'Cr ( VI ) reduction'.replace("( ","(").replace(" )",")").lower()

'cr (vi) reduction'

In [15]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    pred = pred.replace(" - ","-").replace("- ","-").replace(" -","-")
    for annotation in annotations:
        ann = annotation["text"]
        ann = ann.lower()
        if ann in pred:
            count +=1
    if count ==0:
        print(pred)

plant
dysregulation
phagocytosis
phosphorylation
incorporation of
decrease
tolerance
negative
enhanced
uptake
sensitivity
priming
necrotic
modulating
cell
of sti symptoms
touch receptor neurons
vasopressor
growth
tissue colonization
nematicidal
artificial
of bronchiolitis obliterans
freeze
-
inflammatory / proliferation stimulus
enhances
promote plant
probiotic
coagulation
fixation
environmental conditions
virus
balance
mirna biogenesis
promote plant
production of
suppressed
signal
attach
promote
reduced autophagy
biosynthesis
auto
boosted
atmospheric
stress
promote
anti
antioxidant
degradation
plant
total antioxidant
biofilm initiation
sequestration
phenotype
homeostasis
targeted delivery
collembola abundances
wound
enhanced osmotic
solubilizing
tumour
antibiotic
productivity
hyperbiofilm
antipathogenic
stable aggregate
biofilm
acceptors
a relative homeostasis
of pro-inflammatory cytokines
the xylem
salt
rate
reaction
antifungal
plant
processing
growth
2 +
inhibitory
depolymerization


In [37]:
# Create an empty dataframe to store the results
results_df = pd.DataFrame(columns=['NER', 'Prediction'])

ners = ["COMPOUND", "DISEASE", "EFFECT", "ISOLATE", "MEDIUM", "METABOLITE", "ORGANISM",
        "PHENOTYPE", "PROTEIN", "SPECIES", "STRAIN"]

for ner in ners:
    test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
    test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
    merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
    merged.loc[:, "ner"] = ner
    label_ids = merged[["label_ids"]]

    annotations = []
    current_annotation = None

    for j in range(len(df)):
        for i in zip(df.loc[j, "tokens"], df.loc[j, "ner_tags"]):
            if i[1] != "O":
                if current_annotation is None:
                    current_annotation = {"text": i[0]}
                else:
                    current_annotation["text"] += " " + i[0]
            else:
                if current_annotation is not None:
                    annotations.append(current_annotation)
                    current_annotation = None

    if current_annotation is not None:
        annotations.append(current_annotation)

    path = f"../NER_output/{ner}"

    tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path)

    nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
                   aggregation_strategy="average", device=0)

    # Specify the path to the JSONL file
    jsonl_file = f"../NER/{ner}/test.jsonls"

    predictions = []
    # Open the JSONL file
    with jsonlines.open(jsonl_file) as reader:
        # Iterate over each line in the file
        for line in reader:
            for l in line:

                # Get the value from the "text" field
                text = l["data"]["text"]
                # Pass the value to the nlp() function
                predict = nlp(text)
                predictions.append(predict)
    predictions = [p for p in predictions if p]

    # Store the predictions in the dataframe
    for prediction in predictions:
        pred = prediction[0]["word"]
        count = 0
        pred = pred.replace(" - ", "-").replace("- ", "-").replace(" -", "-").lower()
        for annotation in annotations:
            ann = annotation["text"]
            ann = ann.lower()
            if ann in pred:
                count += 1
        if count == 0:
            results_df = pd.concat([results_df, pd.DataFrame({'NER': [ner], 'Prediction': [pred]})], ignore_index=True)

    annotations_set = [i["text"].lower().split(" ") for i in annotations]
    annotations_set = set(itertools.chain.from_iterable(annotations_set))

    predictions_set = [i[0]["word"].lower().replace(" - ", "-").replace("- ", "-").replace(" -", "-").split(" ") for i in predictions]
    predictions_set = set(itertools.chain.from_iterable(predictions_set))

    ann_not_pred = annotations_set.difference(predictions_set)
    pred_not_ann = predictions_set.difference(annotations_set)

    with open(f"ner_annotation/{ner}_ann_not_pred.txt", "w") as f:
        for i in ann_not_pred:
            f.write(i + "\n")
    with open(f"ner_annotation/{ner}_pred_not_ann.txt", "w") as f:
        for i in pred_not_ann:
            f.write(i + "\n")



/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python